The purpose of this file is to show how to read and clean multiple Excel worksheets from a workbook into python

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np 

C:\Users\M.Amer\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#read the dataframe 
bp = pd.read_excel('bp-energy-outlook-2020-summary-tables.xlsx')

Readint eh excel worksheet allows creating a dataframe with index and name of all worksheets inside the workbook as below

In [3]:
# define country list available in the report
# should always be checked in each version to make sure that the countries exist

country_list = ['World',
 'United States',
 'Brazil',
 'EU',
 'Russia',
 'Middle East',
 'Africa',
 'China',
 'India',
 'Other Asia']

In [4]:
sectors_int = ['Oil',
 'Natural gas+',
 'Coal',
 'Nuclear',
 'Hydro',
 'Renewables*',
 'Oil (Mbd)',
 'Natural gas+ (Bcm)',
 'Transport',
 'Industry',
 'Non-combusted',
 'Buildings',
 'Inputs to power',
 'Oil',
 'Natural gas+',
 'Coal',
 'Oil (Mbd)',
 'Natural gas+ (Bcm)',
 'Net CO2 emissions (Mt)']

## Complete function to convert the sheet into a clean form

In [5]:
def clean_table_bp(country):
    
    xls = pd.ExcelFile('bp-energy-outlook-2020-summary-tables.xlsx')
    

        #read the dataframe from the sheet
    df = pd.read_excel(xls, country)

        #adjust column anmes
    df.columns =['sector', '1995', '2000','2005', '2010', '2015', '2018', '1995-2018', 
                         'empty1', '2025 Rapid scenario', '2030 Rapid scenario', '2035 Rapid scenario', 
                         '2040 Rapid scenario', '2045 Rapid scenario', '2050 Rapid scenario', '2018-2050', 'empty2', 
                         '2025 Net Zero scenario', '2030 Net Zero scenario', '2035 Net Zero scenario', '2040 Net Zero scenario', 
                         '2045 Net Zero scenario', '2050 Net Zero scenario', '2018-2050', 'empty3', 
                         '2025 Bussiness as susual', '2030 Bussiness as usual', '2035 Bussiness as usual',
                         '2040 Bussiness as susual', '2045 Bussiness as usual', '2050 Bussiness as usual', '2018-2050']

        #create a column name with the country of interest
    df['country'] = country
        
        
        
        #print('Adjusted the column names and dropped the unncessary cols')

        #choose sectors of interest
    df = df[df['sector'].isin(sectors_int)]
        
    #drop the empty rows
    df.drop(['empty1', 'empty2', 'empty3'], axis = 1, inplace = True)
    
            #adjust sector names to differenctiate between consumption and production of fuels 
    for x in [6,7,8,9,10,11,13,14, 25,26,27,29,30]:
        if x<15:
            df.loc[x,'sector'] = df.loc[x,'sector']+' consumption'
        if x>15:
            df.loc[x,'sector'] = df.loc[x,'sector']+' production'
        
        #print("sector amendments")

        #reset_index
    df = df.reset_index()
    return df

#### Check the code is working successfully

In [6]:
US = clean_table_bp('United States')

In [7]:
US.head()

,index,sector,1995,2000,2005,2010,2015,2018,1995-2018,2025 Rapid scenario,...,2050 Net Zero scenario,2018-2050,2025 Bussiness as susual,2030 Bussiness as usual,2035 Bussiness as usual,2040 Bussiness as susual,2045 Bussiness as usual,2050 Bussiness as usual,2018-2050,country
0,6,Oil consumption,34.58,38.35,40.23,35.56,35.62,36.57,0.002,33.74,...,5.56,-0.057,35.08,33.08,30.83,28.85,26.99,25.34,-0.011,United States
1,7,Natural gas+ consumption,21.53,22.62,21.43,23.33,26.75,29.39,0.014,29.96,...,9.60,-0.034,30.73,31.53,32.46,32.88,32.84,33.40,0.004,United States
2,8,Coal consumption,20.13,22.63,22.85,20.88,15.58,13.27,-0.018,5.63,...,0.02,-0.184,9.59,7.23,5.75,4.47,3.50,2.59,-0.05,United States
3,9,Nuclear consumption,7.09,7.94,7.98,7.98,7.65,7.60,0.003,7.13,...,7.74,0.001,7.13,6.70,5.13,4.12,2.78,0.73,-0.071,United States
4,10,Hydro consumption,3.11,2.73,2.58,2.42,2.25,2.58,-0.008,2.57,...,2.98,0.005,2.54,2.52,2.50,2.48,2.46,2.45,-0.002,United States


## Contstruct a loop to extract data from multiple sheets

In [8]:
# loop over the country list and create a dataframe for each country then append the dataframes

#create an empty dataframe
df_final = pd.DataFrame()

# loop over the country list
for i in country_list:
    
    #perform the function clean_table_bp on each element in the country list
    df = clean_table_bp(i)
    
    # merge the resulting dataframes
    df_final = pd.concat([df, df_final], axis = 0)
    
# return the final dataframe    
df_final
    

,index,sector,1995,2000,2005,2010,2015,2018,1995-2018,2025 Rapid scenario,...,2050 Net Zero scenario,2018-2050,2025 Bussiness as susual,2030 Bussiness as usual,2035 Bussiness as usual,2040 Bussiness as susual,2045 Bussiness as usual,2050 Bussiness as usual,2018-2050,country
0,6,Oil consumption,9.59,11.32,13.00,14.68,16.70,18.19,0.028,19.66,...,5.51,-0.037,20.4,21.38,22.29,22.97,23.47,23.75,0.008,Other Asia
1,7,Natural gas+ consumption,2.98,4.55,6.68,8.16,9.32,9.61,0.052,12.78,...,9.32,-0.001,11.14,12.66,13.75,14.64,15.34,15.75,0.016,Other Asia
2,8,Coal consumption,2.67,3.59,5.21,6.57,7.92,9.68,0.058,10.14,...,1.83,-0.051,12.9,14.95,16.93,18.71,20.26,21.08,0.025,Other Asia
3,9,Nuclear consumption,0.36,0.39,0.41,0.41,0.38,0.34,-0.002,0.48,...,2.49,0.064,0.36,0.36,0.47,0.56,0.67,0.80,0.027,Other Asia
4,10,Hydro consumption,0.92,0.93,1.10,1.37,1.76,2.23,0.039,3.47,...,6.52,0.034,3.37,4.02,4.32,4.60,4.85,5.10,0.026,Other Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,26,Natural gas+ production,75.17,86.49,99.18,113.42,126.05,139.14,0.027,154.49,...,NaN,NaN,153.5,166.27,174.53,180.09,183.77,187.19,0.009,World
15,27,Coal production,94.10,96.48,126.18,150.82,161.68,165.06,0.025,132.93,...,NaN,NaN,155.84,150.40,145.62,137.66,128.82,120.13,-0.01,World
16,29,Oil (Mbd) production,69.21,76.20,83.73,85.59,94.28,97.77,0.015,94.48,...,NaN,NaN,98.17,97.83,96.62,94.05,91.61,89.19,-0.003,World
17,30,Natural gas+ (Bcm) production,2087.93,2402.48,2755.05,3150.69,3501.29,3865.06,0.027,4291.52,...,NaN,NaN,4263.9,4618.66,4847.97,5002.61,5104.60,5199.59,0.009,World
